# Forward Mode Automatic Differentiation
## Operator Overloading Approach

In this notebook, we will implement **Forward Mode** and **Reverse Mode** from **Automatic Differentiation** using the operator overloading method in Python. The core idea is to define a custom `Variable` class that keeps track of both the value and its derivative (the primal and tangent) and overloads arithmetic operations to propagate derivatives according to calculus rules.


## 🧮 The `Variable` Class (Forward Mode)

The `Variable` class in forward mode automatic differentiation represents a value during computation along with its **tangent**, which is the derivative of the function with respect to that variable.

It encapsulates two key components:
- **`primal`**: the actual numerical value of the variable (used to evaluate the function normally)
- **`tangent`**: the value of the derivative of the output with respect to this variable (used to track how a small change in the input affects the output)

In forward mode, we compute both the value and derivative **simultaneously** as we evaluate the function. This is particularly efficient when the number of inputs is small compared to the number of outputs.

### Operator Overloading

We override the standard arithmetic operations (`+`, `-`, `*`, `/`) to support automatic propagation of derivatives. Each operation not only computes the result of the operation but also applies the **chain rule** to update the tangent accordingly.

For instance, if `x` and `y` are `Variable` objects:
- `x * y` will compute:
  - `primal = x.primal * y.primal`
  - `tangent = x.tangent * y.primal + y.tangent * x.primal`

We also support operations between a `Variable` and a plain Python number, making it seamless to mix constants and variables in expressions.

### Elementary Functions

Functions like `sin`, `exp`, and `square` are defined specifically for `Variable` inputs. They compute both:
- the function value (e.g. `sin(x.primal)`)
- the derivative using the chain rule (e.g. `cos(x.primal) * x.tangent` for `sin(x)`)

This setup allows us to compute the derivative of any scalar function programmatically, just by initializing the input variables with appropriate tangent values (typically `1.0` for the variable of interest and `0.0` for others).

Forward mode is ideal when you're interested in how a function changes with respect to **one** input at a time, such as computing directional derivatives or columns of the Jacobian matrix.


In [10]:
import math

class Variable:
    def __init__(self, primal, tangent=0.0):
        self.primal = primal
        self.tangent = tangent

    def __add__(self, other):
        return Variable(self.primal + other.primal, self.tangent + other.tangent)

    def __sub__(self, other):
        return Variable(self.primal - other.primal, self.tangent - other.tangent)

    def __mul__(self, other):
        return Variable(
            self.primal * other.primal,
            self.tangent * other.primal + other.tangent * self.primal
        )

    def __truediv__(self, other):
        return Variable(
            self.primal / other.primal,
            (self.tangent * other.primal - self.primal * other.tangent) / (other.primal**2)
        )
    
    def __radd__(self, other):
        return self + other if isinstance(other, Variable) else Variable(self.primal + other, self.tangent)

    def __rmul__(self, other):
        return self * other if isinstance(other, Variable) else Variable(self.primal * other, self.tangent * other)


    def __repr__(self):
        return f"primal: {self.primal:.3f}, tangent: {self.tangent:.3f}"
        

def sin(x):
    return Variable(math.sin(x.primal), math.cos(x.primal) * x.tangent)

def exp(x):
    epx = math.exp(x.primal)
    return Variable(epx, epx * x.tangent)

def square(x):
    return Variable(x.primal**2, 2 * x.primal * x.tangent)

    


## Example 1: Automatic differentiation in forward mode

Let's calculate the derivative of the function

$$f(x_1, x_2) = \left[\sin\left(\frac{x_1}{x_2}\right) + \frac{x_1}{x_2} - e^{x_2}\right] \cdot \left[\frac{x_1}{x_2} - e^{x_2}\right]$$

using forward mode, at the point $ (x_1, x_2) = (1.5, 0.5) $ first with respect to $ x_1 $, and then with respect to $ x_2 $.



In [11]:
def f(x1, x2):
    return (sin(x1 / x2) + x1 / x2 - exp(x2)) * (x1 / x2 - exp(x2))

### Calculation of $ \frac{\partial f}{\partial x_1} $ at (1.5, 0.5)

In [12]:
x1_val = 1.5
x1_tan = 1.0
x2_val = 0.5
x2_tan = 0.0

x1 = Variable(x1_val, x1_tan)  # ∂x1/∂x1 = 1
x2 = Variable(x2_val, x2_tan)  # ∂x2/∂x1 = 0

y = f(x1, x2)
print(f"f({x1_val:.1f}, {x2_val:.1f}) = {y.primal:.3f}")
print(f"∂f/∂x1 = {y.tangent:.3f}")

f(1.5, 0.5) = 2.017
∂f/∂x1 = 3.012


### Calculation of $ \frac{\partial f}{\partial x_2} $ at (1.5, 0.5)

In [13]:
x1_val = 1.5
x1_tan = 0.0
x2_val = 0.5
x2_tan = 1.0

x1 = Variable(x1_val, x1_tan)  # ∂x1/∂x1 = 0
x2 = Variable(x2_val, x2_tan)  # ∂x2/∂x1 = 1

y = f(x1, x2)
print(f"f({x1_val:.1f}, {x2_val:.1f}) = {y.primal:.3f}")
print(f"∂f/∂x2 = {y.tangent:.3f}")


f(1.5, 0.5) = 2.017
∂f/∂x2 = -13.724


## Example 2: Automatic differentiation in forward mode

Let's calculate the derivative of the function

$$
g(x_1, x_2, x_3, x_4) = (x_2 \sin(x_1) + x_2^2, 2 \, x_3x_4 + x_1) = (r_0, r_1)
$$

using forward mode, at the point $(x_1, x_2, x_3, x_4) = (1.5, 0.5, 2.0, 3.0)$. This example will focus on getting the Jacobian matrix.

In [14]:
def g(x1, x2, x3, x4):
    r0 = x2 * sin(x1) + square(x2)
    r1 = 2 * x3 * x4 + x1
    return [r0, r1]


In [15]:
def compute_jacobian(f, x_vals):
    n_inputs = len(x_vals)
    y_sample = f(*[Variable(val, 0.0) for val in x_vals])
    n_outputs = len(y_sample)
    
    jacobian = []

    for i in range(n_inputs):
        # Set all tangents to 0.0 except for the i-th variable
        x_vars = [Variable(val, 1.0 if j == i else 0.0) for j, val in enumerate(x_vals)]
        y = f(*x_vars)
        jacobian.append([yi.tangent for yi in y])
    
    # Print primal output and the Jacobian in aligned format with square brackets
    print(f"g({', '.join(map(str, x_vals))}) =", [yi.primal for yi in y_sample])
    print("\nJ_g:")
    for row in list(map(list, zip(*jacobian))):
        print(f"[{', '.join(f'{val: .3f}' for val in row)}]")

    return jacobian


In [16]:
x_vals = [1.5, 0.5, 2.0, 3.0]
J = compute_jacobian(g, x_vals)

g(1.5, 0.5, 2.0, 3.0) = [0.7487474933020273, 13.5]

J_g:
[ 0.035,  1.997,  0.000,  0.000]
[ 1.000,  0.000,  6.000,  4.000]


# Reverse Mode

## The `Variable` Class (Reverse Mode)

The `Variable` class in reverse mode automatic differentiation represents a value used during computation and its corresponding **adjoint**, which holds the derivative of a final output with respect to that value.

It encapsulates two key attributes:
- **`primal`**: the actual value of the variable (used during function evaluation)
- **`adjoint`**: the accumulated gradient (derivative of the final result with respect to this variable)

In reverse mode, we evaluate the function **forward** (to compute all intermediate values), and then initiate a **reverse pass** (by calling `.backward()`) from the output back through the computation graph, accumulating derivatives into the `adjoint` of each variable.

Each arithmetic operation constructs a new `Variable` and assigns it a custom `backward()` function, which:
1. Accumulates the incoming gradient into the result variable.
2. Computes the local partial derivatives.
3. Propagates those gradients to the operand variables using their `backward()` methods.

We also define reverse-mode implementations for elementary functions like `sin`, `exp`, and `square`, and include Python operator overloads so expressions like `2 * x` or `x + 3.0` work seamlessly.

This setup allows us to compute gradients of scalar outputs with respect to any number of inputs automatically, just by calling `.backward(1.0)` on the final result.


In [22]:
import math

class Variable:
    def __init__(self, primal, adjoint=0.0):
        self.primal = primal
        self.adjoint = adjoint

    def backward(self, adjoint):
        self.adjoint += adjoint

    def __add__(self, other):
        variable = Variable(self.primal + other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self.backward(adjoint)
            other.backward(adjoint)
        variable.backward = backward
        return variable

    def __sub__(self, other):
        variable = Variable(self.primal - other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self.backward(adjoint)
            other.backward(-adjoint)
        variable.backward = backward
        return variable

    def __mul__(self, other):
        variable = Variable(self.primal * other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self.backward(adjoint * other.primal)
            other.backward(adjoint * self.primal)
        variable.backward = backward
        return variable

    def __truediv__(self, other):
        variable = Variable(self.primal / other.primal)
        def backward(adjoint):
            variable.adjoint += adjoint
            self.backward(adjoint * (1.0 / other.primal))
            other.backward(adjoint * (-self.primal / (other.primal ** 2)))
        variable.backward = backward
        return variable
    
    def __radd__(self, other):
        return self + Variable(other)

    def __rsub__(self, other):
        return Variable(other) - self

    def __rmul__(self, other):
        return self * Variable(other)

    def __rtruediv__(self, other):
        return Variable(other) / self


    def __repr__(self):
        return f"primal: {self.primal}, adjoint: {self.adjoint}"
 

def sin(var):
    result = Variable(math.sin(var.primal))
    def backward(adjoint):
        result.adjoint += adjoint
        var.backward(adjoint * math.cos(var.primal))
    result.backward = backward
    return result

def exp(var):
    result = Variable(math.exp(var.primal))
    def backward(adjoint):
        result.adjoint += adjoint
        var.backward(adjoint * result.primal)
    result.backward = backward
    return result

def square(var):
    result = Variable(var.primal ** 2)
    def backward(adjoint):
        result.adjoint += adjoint
        var.backward(adjoint * 2 * var.primal)
    result.backward = backward
    return result



## Example 1: Automatic differentiation in reverse mode

Let's calculate the derivative of the function

$$
f(x_1, x_2) = \left[\sin\left(\frac{x_1}{x_2}\right) + \frac{x_1}{x_2} - e^{x_2}\right] \cdot \left[\frac{x_1}{x_2} - e^{x_2}\right]
$$

using forward mode, at the point $ (x_1, x_2) = (1.5, 0.5) $ first with respect to $ x_1 $, and then with respect to $ x_2 $.



In [23]:
x1 = Variable(1.5)
x2 = Variable(0.5)

y = f(x1, x2)
y.backward(1.0)

print(f"f({x1.primal:.1f}, {x2.primal:.1f}) = {y.primal:.3f}")
print(f"∂y/∂x1 = {x1.adjoint}")
print(f"∂y/∂x2 = {x2.adjoint}")


f(1.5, 0.5) = 2.017
∂y/∂x1 = 3.011843327673907
∂y/∂x2 = -13.723961509314076


## Example 2: Automatic differentiation in reverse mode

Let's calculate the derivative of the function

$$
g(x_1, x_2, x_3, x_4) = (x_2 \sin(x_1) + x_2^2, 2 \, x_3x_4 + x_1) = (r_0, r_1)
$$

using reverse mode, at the point $(x_1, x_2, x_3, x_4) = (1.5, 0.5, 2.0, 3.0)$. This example will focus on getting the Jacobian matrix.

In [24]:
def g(x1, x2, x3, x4):
    r0 = x2 * sin(x1) + square(x2)
    r1 = 2 * x3 * x4 + x1
    return [r0, r1]

In [25]:
def compute_jacobian_reverse(f, x_vals):
    n_inputs = len(x_vals)

    # Wrap raw inputs as Variables (fresh copies for each reverse pass)
    def make_vars():
        return [Variable(val) for val in x_vals]

    # Forward pass to get outputs and output count
    vars = make_vars()
    outputs = f(*vars)
    n_outputs = len(outputs)

    jacobian = []

    for i in range(n_outputs):
        # Reset adjoints (fresh inputs and outputs every time)
        vars = make_vars()
        outputs = f(*vars)

        # Start reverse pass for output i
        outputs[i].backward(1.0)

        # Collect ∂output_i/∂xj (j = 0 to n_inputs-1)
        jacobian.append([var.adjoint for var in vars])

    # Print formatted Jacobian
    print(f"g({', '.join(map(str, x_vals))}) =", [y.primal for y in outputs])
    print("\nJ_g:")
    for row in jacobian:
        print(f"[{', '.join(f'{val: .3f}' for val in row)}]")

    return jacobian



In [26]:
x_vals = [1.5, 0.5, 2.0, 3.0]
J = compute_jacobian_reverse(g, x_vals)


g(1.5, 0.5, 2.0, 3.0) = [0.7487474933020273, 13.5]

J_g:
[ 0.035,  1.997,  0.000,  0.000]
[ 1.000,  0.000,  6.000,  4.000]
